In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [30]:
def clipped_normal_distriution(x0,sigma):
    value = np.random.normal(x0,sigma)
    return(value if value>=0 else clipped_normal_distriution(x0,sigma))

class social_simulation:
    """meritocracy game
    """
    
    def __init__(self,corporate_size=1000,group_size=5,group_competition_number=4):
        """initialize the simulation, by setting global parameters like the corporate and group size
        (corporate size should be a multiple of group size)
        """
        self._index_id=0
        self.corporate_size=corporate_size
        self.group_size=group_size
        self.number_of_groups=int(self.corporate_size/self.group_size)
        self.group_competition_number=group_competition_number
        self._check_parameter_consistency()

        # further default parameters
        self.random_performance_deviation=0.25

        self.overachiever_performance_bonus_factor=2
        self.charismatc_idiot_performance_malus_factor=0.5
        
        self.overachiever_group_malus=-0.25
        self.charismatic_idiot_group_bonus=0.25

        self.charismatic_idiot_promotion_bonus=2

        
    def _check_parameter_consistency(self):
        """assert that corporate_size, group_size and group_competition_number fulfil all conditions of multiples
        """
        if self.corporate_size%self.group_size != 0:
            print("corporate size "+str(self.corporate_size)+
                  " is not divisible into an integer number of groups of size "+str(self.group_size))
            adjusted_corp_size=int(np.ceil(self.corporate_size/self.group_size)*self.group_size) 
            print("adjusted corporate size: "+str(adjusted_corp_size))
            self.corporate_size=adjusted_corp_size
            self.number_of_groups=int(self.corporate_size/self.group_size)
            print()
            
        if self.number_of_groups%self.group_competition_number !=0:
            print("number of groups "+str(self.number_of_groups)+
                  " is not divisible into an integer number of group competitions each containing "+
                  str(self.group_competition_number)+ " groups")
            adjusted_num_groups=int(np.ceil(self.number_of_groups/self.group_competition_number)*self.group_competition_number)
            adjusted_corp_size=int(adjusted_num_groups*self.group_size)
            print("adjusted number of groups: "+str(adjusted_num_groups))
            print("leading to adjusted corporate size: "+str(adjusted_corp_size))
            self.corporate_size=adjusted_corp_size
            self.number_of_groups=adjusted_num_groups
            print()
                    
    @staticmethod
    def _get_personality_from_index(index):
        """helper function to get personality distribution from random integers with uniform probability
        0,1 (50%) -> normal , 2 (25%) -> overachiever , 3 (25%) -> charismatic idiot
        """
        if index <2:
            personality="normal"
        elif index==2:
            personality="overachiever"
        elif index==3:
            personality="charismatic idiot"
        return personality

    @staticmethod
    def _get_performance_scores(random_performance_deviation,number_of_trainees):
        return [clipped_normal_distriution(1,random_performance_deviation) for i in range(number_of_trainees)]

    def create_trainees(self,number_of_trainees,random_performance_deviation):
        """the given number of trainees is created, by three lists containing:
        the trainee ID, the trainee personality and the trainee performance
        """
        # trainee index / ID
        trainee_id=[self._index_id+i for i in range(self._index_id+number_of_trainees)]
        self._index_id += number_of_trainees

        # create personalities
        personality_index=np.random.randint(0,4,number_of_trainees)
        personality=[self._get_personality_from_index(i) for i in personality_index]

        # create corresponding performance scores
        performance=self._get_performance_scores(random_performance_deviation,number_of_trainees)
        
        for i in range(len(personality)):
            if personality[i]=="normal":
                pass
            elif personality[i]=="overachiever":
                performance[i] *= self.overachiever_performance_bonus_factor
            elif personality[i]=="charismatic idiot":
                performance[i] *= self.charismatc_idiot_performance_malus_factor

        return trainee_id,personality,performance
                
    def initialize_groups(self):
        """Create the goup structure as a list of lists, with each list having the length of the group size,
        containing the trainee IDs belonging to that group. Additionally a list for look-up is created,
        which links each trainee ID to the corresponding group index.
        """
        self.group_lookup = [i//self.group_size for i in range(self.corporate_size)]
        self.groups=[]
        for i in range(self.corporate_size):
            if i%self.group_size == 0:
                self.groups.append([])
                self.groups[-1].append(self.trainee_id[i])

    def initialize_corporation(self):
        trainee_id,personality,performance=self.create_trainees(self.corporate_size,self.random_performance_deviation)
        self.trainee_id=trainee_id
        self.personality=personality
        self.performance=performance
        self.initialize_groups()

        self.entry_time=[0 for i in range(self.corporate_size)]
        self.promoted=[None for i in range(self.corporate_size)]
        self.time_step=0
  
    def calculate_group_performance(self,group):
        personalities_in_group=np.array([self.personality[i] for i in group])    
        number_of_overachievers=sum(personalities_in_group=="overachiever")
        number_of_charismatic_idiots=sum(personalities_in_group=="charismatic idiot")
        performance_sum=sum([self.performance[i] for i in group]) 
        bonus_sum=(len(group)-1)*number_of_charismatic_idiots* self.charismatic_idiot_group_bonus
        malus_sum=(len(group)-1)*number_of_overachievers*self.overachiever_group_malus
        return performance_sum + bonus_sum + malus_sum
            
    def select_promotion(self,group):
        promotion_scores=np.empty(len(group))
        for counter,i in enumerate(group):
            if self.personality[i]=="charismatic idiot":
                promotion_scores[counter]=self.performance[i] +self.charismatic_idiot_promotion_bonus
            else:
                promotion_scores[counter]=self.performance[i]
        promotion_group_index=np.argmax(promotion_scores)
        promotion_global_index=group[promotion_group_index]
        return promotion_group_index,promotion_global_index
            
    def group_competition_round(self):
        #random order of groups
        random_for_sorting=np.random.uniform(size=self.number_of_groups)
        competition_order=np.argsort(random_for_sorting)

        #execute group competitions
        number_of_competitions=int(self.number_of_groups/self.group_competition_number)
        winning_groups=[]
        for i in range(number_of_competitions):
            competing_groups=[]
            group_scores=np.empty(self.group_competition_number)
            for j in range(self.group_competition_number):
                competing_groups.append(competition_order[i*self.group_competition_number+j])
                group_scores[j]=self.calculate_group_performance(self.groups[competing_groups[-1]])
            winning_index=np.argmax(group_scores)
            winning_groups.append(competing_groups[winning_index])
        return winning_groups
        
    def promotion_and_substitution_round(self,winning_groups):
        self.time_step+=1

        number_of_new_trainees=len(winning_groups)
        trainee_id,personality,performance=self.create_trainees(number_of_new_trainees,self.random_performance_deviation)
        self.trainee_id+=trainee_id
        self.personality+=personality
        self.performance+=performance
        self.entry_time+=[self.time_step for i in range(number_of_new_trainees)]
        self.promoted+=[None for i in range(number_of_new_trainees)]

        for i in winning_groups:
            winner_local,winner_global=self.select_promotion(self.groups[i])
            self.promoted[winner_global]=self.time_step
            self.groups[winner_local]=trainee_id[i]

In [31]:
a=social_simulation(group_size=6)
a.initialize_corporation()
winners=a.group_competition_round()
a.promotion_and_substitution_round(winners)

corporate size 1000 is not divisible into an integer number of groups of size 6
adjusted corporate size: 1002

number of groups 167 is not divisible into an integer number of group competitions each containing 4 groups
adjusted number of groups: 168
leading to adjusted corporate size: 1008

